In [209]:
import pandas as pd
import re
from collections import Counter

In [193]:
df1 = pd.read_excel('naver_wiki_product_ver2.xlsx')
df1

,index,제품명,향기,단맛,신맛,청량감,바디감,목넘김,균형감,주종,도수,용량,가격,원재료,제조사,키워드(thesool),어울리는음식(thesool)
0,0,1000억막걸리프리바이오,NaN,3.0,4.0,3.0,3,NaN,NaN,탁주,5%,750ml,3500,"쌀(국내산),밀(국내산),누룩,정제수",(주)국순당,NaN,NaN
1,1,1000억유산균막걸리,NaN,3.0,5.0,5.0,3,NaN,NaN,탁주,5%,750ml,3200,"쌀(국내산),밀(국내산),누룩,정제수",(주)국순당,저도수,"적당한 감칠맛과 청량미가 있어 전, 회무침과 잘 어울린다."
2,2,1932새싹땅콩햅쌀막걸리,NaN,2.0,2.0,3.0,5,NaN,NaN,생막걸리,6%,900ml,NaN,"쌀(국내산),땅콩농축액,입국,효모,고과당,정제효소제,정제수",(주)1932포천일동막걸리,NaN,NaN
3,3,1932새싹땅콩흑미막걸리,NaN,3.0,1.0,1.0,4,NaN,NaN,생막걸리,6%,900ml,NaN,"쌀(국내산),흑미(15%),땅콩농축액,입국,효모,고과당,정제효소제,정제수",(주)1932포천일동막걸리,NaN,마늘새우
4,4,33JU,4.0,3.0,NaN,NaN,1,1.0,NaN,일반증류주,18%,500ml/200ml,NaN,"산양산삼,주정,결정과당,효소처리스테비아,에리스리톨,호박산,꿀",농업회사법인33가주식회사,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,528,회곡생동동주,NaN,3.0,2.0,3.0,3,NaN,NaN,탁주,7%,"1,200ml",1300,"쌀,입국,누룩,아스파탐,효모,정제수,아세설팜칼륨",회곡양조장,NaN,NaN
529,529,회곡생막걸리,NaN,2.0,2.0,2.0,4,NaN,NaN,탁주,6%,750ml,1300,"쌀,입국,누룩,아스파탐,효모,정제수,아세설팜칼륨",회곡양조장,NaN,NaN
530,530,흑돈주,2.0,NaN,NaN,NaN,2,3.0,NaN,기타주류,16%,365ml,12000,"황칠나무(제주산),정제수,포도당,효모",왕지케양조장(제주본초협동조합),NaN,NaN
531,531,희양산막걸리15,NaN,3.0,4.0,1.0,5,NaN,NaN,탁주,15%,500ml,9500,"쌀(국내산),누룩,정제수",두술도가,무감미료,매콤한 볶음류의 음식


# 주종 처리

In [194]:
# 탁주, 약/청주, 과실주, 증류주 , 기타주류(리큐르는 여기포함) 로 구분.

# 살균약주, 약주(생약주) -> 약주
df1.loc[df1['주종'] == '살균약주', ['주종']] = '약주'
df1.loc[df1['주종'] == '약주(생약주)', ['주종']] = '약주'
# 약주, 청주 -> 약주/청주
df1.loc[df1['주종'] == '약주', ['주종']] = '약주/청주'
df1.loc[df1['주종'] == '청주', ['주종']] = '약주/청주'

# 살균탁주, 생탁주, 생막걸리, 전통수제탁주 -> 탁주
df1.loc[df1['주종'] == '살균탁주', ['주종']] = '탁주'
df1.loc[df1['주종'] == '생탁주', ['주종']] = '탁주'
df1.loc[df1['주종'] == '생막걸리', ['주종']] = '탁주'
df1.loc[df1['주종'] == '전통수제탁주', ['주종']] = '탁주'

# 과실주(포도) -> 과실주
df1.loc[df1['주종'] == '과실주(포도)', ['주종']] = '과실주'

# 일반증류주, 증류식소주, 소주, 브랜디 -> 증류주
df1.loc[df1['주종'] == '일반증류주', ['주종']] = '증류주'
df1.loc[df1['주종'] == '증류식소주', ['주종']] = '증류주'
df1.loc[df1['주종'] == '소주', ['주종']] = '증류주'
df1.loc[df1['주종'] == '브랜디', ['주종']] = '증류주'

# 리큐르 -> 기타주류
df1.loc[df1['주종'] == '리큐르', ['주종']] = '기타주류'

In [195]:
df1['주종'].value_counts()

탁주       165
과실주      117
증류주      104
약주/청주     90
기타주류      57
Name: 주종, dtype: int64

# 도수 정리

In [196]:
# 도수 컬럼 숫자만 남기기
df1['도수'] = df1['도수'].apply(lambda x : re.sub('\%', "", x ))

In [197]:
df1

,index,제품명,향기,단맛,신맛,청량감,바디감,목넘김,균형감,주종,도수,용량,가격,원재료,제조사,키워드(thesool),어울리는음식(thesool)
0,0,1000억막걸리프리바이오,NaN,3.0,4.0,3.0,3,NaN,NaN,탁주,5,750ml,3500,"쌀(국내산),밀(국내산),누룩,정제수",(주)국순당,NaN,NaN
1,1,1000억유산균막걸리,NaN,3.0,5.0,5.0,3,NaN,NaN,탁주,5,750ml,3200,"쌀(국내산),밀(국내산),누룩,정제수",(주)국순당,저도수,"적당한 감칠맛과 청량미가 있어 전, 회무침과 잘 어울린다."
2,2,1932새싹땅콩햅쌀막걸리,NaN,2.0,2.0,3.0,5,NaN,NaN,탁주,6,900ml,NaN,"쌀(국내산),땅콩농축액,입국,효모,고과당,정제효소제,정제수",(주)1932포천일동막걸리,NaN,NaN
3,3,1932새싹땅콩흑미막걸리,NaN,3.0,1.0,1.0,4,NaN,NaN,탁주,6,900ml,NaN,"쌀(국내산),흑미(15%),땅콩농축액,입국,효모,고과당,정제효소제,정제수",(주)1932포천일동막걸리,NaN,마늘새우
4,4,33JU,4.0,3.0,NaN,NaN,1,1.0,NaN,증류주,18,500ml/200ml,NaN,"산양산삼,주정,결정과당,효소처리스테비아,에리스리톨,호박산,꿀",농업회사법인33가주식회사,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,528,회곡생동동주,NaN,3.0,2.0,3.0,3,NaN,NaN,탁주,7,"1,200ml",1300,"쌀,입국,누룩,아스파탐,효모,정제수,아세설팜칼륨",회곡양조장,NaN,NaN
529,529,회곡생막걸리,NaN,2.0,2.0,2.0,4,NaN,NaN,탁주,6,750ml,1300,"쌀,입국,누룩,아스파탐,효모,정제수,아세설팜칼륨",회곡양조장,NaN,NaN
530,530,흑돈주,2.0,NaN,NaN,NaN,2,3.0,NaN,기타주류,16,365ml,12000,"황칠나무(제주산),정제수,포도당,효모",왕지케양조장(제주본초협동조합),NaN,NaN
531,531,희양산막걸리15,NaN,3.0,4.0,1.0,5,NaN,NaN,탁주,15,500ml,9500,"쌀(국내산),누룩,정제수",두술도가,무감미료,매콤한 볶음류의 음식


# 원재료 키워드화

In [198]:
df_2nd = df1.copy()
df_2nd

,index,제품명,향기,단맛,신맛,청량감,바디감,목넘김,균형감,주종,도수,용량,가격,원재료,제조사,키워드(thesool),어울리는음식(thesool)
0,0,1000억막걸리프리바이오,NaN,3.0,4.0,3.0,3,NaN,NaN,탁주,5,750ml,3500,"쌀(국내산),밀(국내산),누룩,정제수",(주)국순당,NaN,NaN
1,1,1000억유산균막걸리,NaN,3.0,5.0,5.0,3,NaN,NaN,탁주,5,750ml,3200,"쌀(국내산),밀(국내산),누룩,정제수",(주)국순당,저도수,"적당한 감칠맛과 청량미가 있어 전, 회무침과 잘 어울린다."
2,2,1932새싹땅콩햅쌀막걸리,NaN,2.0,2.0,3.0,5,NaN,NaN,탁주,6,900ml,NaN,"쌀(국내산),땅콩농축액,입국,효모,고과당,정제효소제,정제수",(주)1932포천일동막걸리,NaN,NaN
3,3,1932새싹땅콩흑미막걸리,NaN,3.0,1.0,1.0,4,NaN,NaN,탁주,6,900ml,NaN,"쌀(국내산),흑미(15%),땅콩농축액,입국,효모,고과당,정제효소제,정제수",(주)1932포천일동막걸리,NaN,마늘새우
4,4,33JU,4.0,3.0,NaN,NaN,1,1.0,NaN,증류주,18,500ml/200ml,NaN,"산양산삼,주정,결정과당,효소처리스테비아,에리스리톨,호박산,꿀",농업회사법인33가주식회사,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,528,회곡생동동주,NaN,3.0,2.0,3.0,3,NaN,NaN,탁주,7,"1,200ml",1300,"쌀,입국,누룩,아스파탐,효모,정제수,아세설팜칼륨",회곡양조장,NaN,NaN
529,529,회곡생막걸리,NaN,2.0,2.0,2.0,4,NaN,NaN,탁주,6,750ml,1300,"쌀,입국,누룩,아스파탐,효모,정제수,아세설팜칼륨",회곡양조장,NaN,NaN
530,530,흑돈주,2.0,NaN,NaN,NaN,2,3.0,NaN,기타주류,16,365ml,12000,"황칠나무(제주산),정제수,포도당,효모",왕지케양조장(제주본초협동조합),NaN,NaN
531,531,희양산막걸리15,NaN,3.0,4.0,1.0,5,NaN,NaN,탁주,15,500ml,9500,"쌀(국내산),누룩,정제수",두술도가,무감미료,매콤한 볶음류의 음식


In [199]:
# 원재료 컬럼 1차전처리
df_2nd['원재료'] = df_2nd['원재료'].apply(lambda x : re.sub(r'\([^)]*\)', "", x ))
df_2nd['원재료'] = df_2nd['원재료'].apply(lambda x : re.sub(r'[\d\%\.]', "", x ))
df_2nd['원재료'] = df_2nd['원재료'].apply(lambda x : re.sub('포도당', "", x ))    # 포도에서 검색되지 않도록

In [200]:
# 원재료키워드 컬럼 추가
df_2nd.insert(14, '원재료키워드', '')

In [201]:
df_2nd

,index,제품명,향기,단맛,신맛,청량감,바디감,목넘김,균형감,주종,도수,용량,가격,원재료,원재료키워드,제조사,키워드(thesool),어울리는음식(thesool)
0,0,1000억막걸리프리바이오,NaN,3.0,4.0,3.0,3,NaN,NaN,탁주,5,750ml,3500,"쌀,밀,누룩,정제수",,(주)국순당,NaN,NaN
1,1,1000억유산균막걸리,NaN,3.0,5.0,5.0,3,NaN,NaN,탁주,5,750ml,3200,"쌀,밀,누룩,정제수",,(주)국순당,저도수,"적당한 감칠맛과 청량미가 있어 전, 회무침과 잘 어울린다."
2,2,1932새싹땅콩햅쌀막걸리,NaN,2.0,2.0,3.0,5,NaN,NaN,탁주,6,900ml,NaN,"쌀,땅콩농축액,입국,효모,고과당,정제효소제,정제수",,(주)1932포천일동막걸리,NaN,NaN
3,3,1932새싹땅콩흑미막걸리,NaN,3.0,1.0,1.0,4,NaN,NaN,탁주,6,900ml,NaN,"쌀,흑미,땅콩농축액,입국,효모,고과당,정제효소제,정제수",,(주)1932포천일동막걸리,NaN,마늘새우
4,4,33JU,4.0,3.0,NaN,NaN,1,1.0,NaN,증류주,18,500ml/200ml,NaN,"산양산삼,주정,결정과당,효소처리스테비아,에리스리톨,호박산,꿀",,농업회사법인33가주식회사,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,528,회곡생동동주,NaN,3.0,2.0,3.0,3,NaN,NaN,탁주,7,"1,200ml",1300,"쌀,입국,누룩,아스파탐,효모,정제수,아세설팜칼륨",,회곡양조장,NaN,NaN
529,529,회곡생막걸리,NaN,2.0,2.0,2.0,4,NaN,NaN,탁주,6,750ml,1300,"쌀,입국,누룩,아스파탐,효모,정제수,아세설팜칼륨",,회곡양조장,NaN,NaN
530,530,흑돈주,2.0,NaN,NaN,NaN,2,3.0,NaN,기타주류,16,365ml,12000,"황칠나무,정제수,,효모",,왕지케양조장(제주본초협동조합),NaN,NaN
531,531,희양산막걸리15,NaN,3.0,4.0,1.0,5,NaN,NaN,탁주,15,500ml,9500,"쌀,누룩,정제수",,두술도가,무감미료,매콤한 볶음류의 음식


In [203]:
# 전체제품 원재료 확인
ingredients = []
df_2nd['원재료'].apply(lambda x : ingredients.extend(re.split(',', x)))
result = Counter(ingredients)
print(result)

Counter({'정제수': 300, '누룩': 185, '쌀': 169, '효모': 134, '찹쌀': 52, '국내산쌀': 46, '백미': 41, '아스파탐': 37, '국': 35, '주정': 33, '포도': 32, '젖산': 27, '멥쌀': 26, '구연산': 26, '입국': 25, '정제효소': 22, '소맥분': 20, '오미자': 19, '정제효소제': 16, '국내산찹쌀': 16, '과당': 15, '복분자': 15, '물엿': 14, '전통누룩': 14, '종국': 13, '효소': 12, '효모등': 12, '고과당': 11, '조제종국': 11, '보리': 10, '사과': 10, '구기자': 9, '국내산산머루': 9, '산양산삼': 8, '꿀': 8, '설탕': 8, '전분당': 8, '밀': 7, '벌꿀': 7, '올리고당': 7, '무수아황산': 7, '감': 6, '백설탕': 6, '곡자': 6, '팽화미': 6, '밀함유': 5, '인삼': 5, '생강': 5, '감초': 5, '밀가루': 5, '양조용포도': 5, '아스파탐등': 5, '정제주정': 5, '증류원액': 4, '재리종국화꽃': 4, '진달래': 4, '재래종솔잎': 4, '정제주': 4, '고구마': 4, '정제수등': 4, '밀누룩': 4, '효소제': 4, '과당등': 4, '죽력': 4, '거봉포도원액': 4, '우리밀누룩': 4, '물엿등': 4, '': 4, '젖산등': 4, '산수유': 4, '증류주': 4, '사과농축액': 4, '국내산포도': 4, '햅쌀': 4, '조효소제': 4, '물': 4, '쌀가루': 4, '홍삼농축액': 4, '국내산찹살': 4, '향온곡': 4, '아황산염': 4, '소르빈산염': 4, '효소처리스테비아': 3, '조효소재': 3, '정제효소등': 3, '오가피': 3, '멥살': 3, '맥아': 3, '호프펠렛': 3, '복분자과실': 3, '아세설팜칼륨': 3, '메타중아황산칼륨': 3, '옥수수전분': 3, 

In [211]:
# 전 품목 원재료에서 2개 이상 반복되는 단어들 중 선정.
ing_keywords = ['쌀', '찹쌀', '포도', '오미자', '복분자', '머루', '사과',
                '보리', '구기자', '꿀', '삼', '밀', '오디', '생강', '감초',
                '꽃', '솔잎', '고구마', '귤', '밤', '산수유', '땅콩', '오가피',
                '다래', '계피', '버섯', '감자', '복숭아', '약재', '베리']

In [212]:
# 원재료키워드리스트 저장
df_ing_keywords = pd.DataFrame(ing_keywords)
df_ing_keywords.to_csv('ingredients_keywords.csv')

In [205]:
for row in range(0, len(df_2nd['원재료'])):    
    list_ing_key = []
    for index, value in enumerate(ing_keywords):
        if ing_keywords[index] in df_2nd['원재료'][row]:
            list_ing_key.append(value)
    df_2nd['원재료키워드'][row] = list_ing_key

C:\Users\X1Carbon\AppData\Local\Temp\ipykernel_6312\2364690639.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2nd['원재료키워드'][row] = list_ing_key


In [192]:
# 리스트를 문자열로 변환하는 함수 (데이터프레임 만들고 나서 사용할 예정)
def list_changer(list):
    st = ''
    for word in list:
        st = st + word + ','
    return st

In [206]:
# 리스트껍데기 벗기기
df_2nd['원재료키워드'] = df_2nd['원재료키워드'].apply(lambda x : list_changer(x))

In [207]:
df_2nd

,index,제품명,향기,단맛,신맛,청량감,바디감,목넘김,균형감,주종,도수,용량,가격,원재료,원재료키워드,제조사,키워드(thesool),어울리는음식(thesool)
0,0,1000억막걸리프리바이오,NaN,3.0,4.0,3.0,3,NaN,NaN,탁주,5,750ml,3500,"쌀,밀,누룩,정제수","쌀,밀,",(주)국순당,NaN,NaN
1,1,1000억유산균막걸리,NaN,3.0,5.0,5.0,3,NaN,NaN,탁주,5,750ml,3200,"쌀,밀,누룩,정제수","쌀,밀,",(주)국순당,저도수,"적당한 감칠맛과 청량미가 있어 전, 회무침과 잘 어울린다."
2,2,1932새싹땅콩햅쌀막걸리,NaN,2.0,2.0,3.0,5,NaN,NaN,탁주,6,900ml,NaN,"쌀,땅콩농축액,입국,효모,고과당,정제효소제,정제수","쌀,땅콩,",(주)1932포천일동막걸리,NaN,NaN
3,3,1932새싹땅콩흑미막걸리,NaN,3.0,1.0,1.0,4,NaN,NaN,탁주,6,900ml,NaN,"쌀,흑미,땅콩농축액,입국,효모,고과당,정제효소제,정제수","쌀,땅콩,",(주)1932포천일동막걸리,NaN,마늘새우
4,4,33JU,4.0,3.0,NaN,NaN,1,1.0,NaN,증류주,18,500ml/200ml,NaN,"산양산삼,주정,결정과당,효소처리스테비아,에리스리톨,호박산,꿀","꿀,삼,",농업회사법인33가주식회사,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,528,회곡생동동주,NaN,3.0,2.0,3.0,3,NaN,NaN,탁주,7,"1,200ml",1300,"쌀,입국,누룩,아스파탐,효모,정제수,아세설팜칼륨","쌀,",회곡양조장,NaN,NaN
529,529,회곡생막걸리,NaN,2.0,2.0,2.0,4,NaN,NaN,탁주,6,750ml,1300,"쌀,입국,누룩,아스파탐,효모,정제수,아세설팜칼륨","쌀,",회곡양조장,NaN,NaN
530,530,흑돈주,2.0,NaN,NaN,NaN,2,3.0,NaN,기타주류,16,365ml,12000,"황칠나무,정제수,,효모",,왕지케양조장(제주본초협동조합),NaN,NaN
531,531,희양산막걸리15,NaN,3.0,4.0,1.0,5,NaN,NaN,탁주,15,500ml,9500,"쌀,누룩,정제수","쌀,",두술도가,무감미료,매콤한 볶음류의 음식


In [208]:
# csv 파일로 저장
df_2nd.to_csv('naver_wiki_product_ver3.csv')